## Helper Functions

In [20]:
import operator
from math import log

def count_rows(array, column, value):
    count = 0
    for row in array:
        if row[column] == value:
            count += 1
    return count

def get_dominant_val(y_vals):
    possibilities = {}
    for row in y_vals:
        value = row[0]
        if value not in possibilities:
            possibilities[value] = 1
        else:
            possibilities[value] += 1
    dominant_val = max(possibilities.iteritems(), key=operator.itemgetter(1))[0]
    return dominant_val

def import_csv(file_name):
    csv_file = open(file_name, "r")
    csv_str = csv_file.read()
    csv_array = csv_str.split("\n")
    csv_data = []
    for row in csv_array:
        curr_array = row.split(",")
        if len(curr_array):
            csv_data.append(curr_array)
    return csv_data


def split_col(col_num, array):
    array1 = []
    array2 = []
    for row in array:
        array1_row = []
        array2_row = []
        for i, item in enumerate(row):   
            if i < col_num:
                array1_row.append(item)
            else:
                array2_row.append(item)
        array1.append(array1_row)
        array2.append(array2_row)
    return array1, array2

def split_attr(x_vals, y_vals, attr_num, value):
    x1 = []
    x2 = []
    y1 = []
    y2 = []
    for i, row in enumerate(x_vals):
        if row[attr_num] == value:
            x1.append(row)
            y1.append(y_vals[i])
        else:
            x2.append(row)
            y2.append(y_vals[i])
    return x1, x2, y1, y2

def get_col(array_of_arrays, col_num):
    col = []
    for row in array_of_arrays:
        col.append(row[col_num])
    return col

# Import Data

Guide: https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/
Taken from: https://archive.ics.uci.edu/ml/datasets/iris

_"The data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant. One class is linearly separable from the other 2; the latter are NOT linearly separable from each other."_

In [21]:
iris_data = import_csv("iris.csv")
iris_x, iris_y = split_col(len(iris_data[0])-1, iris_data)

print("Attributes:\n[sepal-length, sepal-width, petal-length, petal-width, class]\n")
print("Iris-setosa example:\n{ex}\n".format(ex=str(iris_x[0]) + ": " + str(iris_y[0])))
print("Iris-versicolor example:\n{ex}\n".format(ex=str(iris_x[50]) + ": " + str(iris_y[50])))
print("Iris-virginica example:\n{ex}\n".format(ex=str(iris_x[100]) + ": " + str(iris_y[100])))
print("Data Length: {data_len}\n".format(data_len=len(iris_data)))


Attributes:
[sepal-length, sepal-width, petal-length, petal-width, class]

Iris-setosa example:
['5.1', '3.5', '1.4', '0.2']: ['Iris-setosa']

Iris-versicolor example:
['7.0', '3.2', '4.7', '1.4']: ['Iris-versicolor']

Iris-virginica example:
['6.3', '3.3', '6.0', '2.5']: ['Iris-virginica']

Data Length: 150



In [42]:
split_val = 100
votes_data = import_csv("house-votes.csv")
votes_y, votes_x = split_col(1, votes_data)
votes_train_y = votes_y[:split_val]
votes_train_x = votes_x[:split_val]
votes_test_y = votes_y[split_val:]
votes_test_x = votes_x[split_val:]

print("Example 1:\n{ex}\n".format(ex=str(votes_train_x[0]) + ": " + str(votes_train_y[0])))
print("Example 2:\n{ex}\n".format(ex=str(votes_train_x[50]) + ": " + str(votes_train_y[50])))
print("Example 3:\n{ex}\n".format(ex=str(votes_train_x[100]) + ": " + str(votes_train_y[100])))

Example 1:
['n', 'y', 'n', 'y', 'y', 'y', 'n', 'n', 'n', 'y', '?', 'y', 'y', 'y', 'n', 'y']: ['republican']

Example 2:
['y', 'y', 'y', 'n', 'n', 'n', 'y', 'y', 'y', 'n', 'y', 'n', 'n', 'n', 'y', 'y']: ['democrat']



IndexError: list index out of range

# Node Class

In [43]:
class Node:
    def __init__(self, attribute=None, split_value=None, left_branch=None, right_branch=None):
        self.attribute = attribute
        self.split_value = split_value
        self.left_branch = left_branch
        self.right_branch = right_branch
        
    def predict(self, x_vals):
        curr_split = x_vals[self.attribute]
        if curr_split == self.split_value:
            if isinstance(self.right_branch, Node):
                return self.right_branch.predict(x_vals)
            else:
                return self.right_branch
        else:
            if isinstance(self.left_branch, Node):
                return self.left_branch.predict(x_vals)
            else:
                return self.left_branch            

# Entropy

In [44]:
def entropy(x_vals, y_vals, attribute, value):
    right_x, left_x, right_y, left_y = split_attr(x_vals, y_vals, attribute, value)
    y_possible = {}
    for row in right_y:
        possibility = row[0]
        if possibility not in y_possible:
            y_possible[possibility] = float(count_rows(right_y, 0, possibility)) / count_rows(y_vals, 0, possibility)
    entropy = 0
    log2=lambda x:log(x)/log(2)  
    for possibility in y_possible:
        ratio = y_possible[possibility]
        print("Ratio: " + str(ratio))
        entropy += ratio*log2(ratio)
    return entropy * -1

# Tree Building 

In [45]:
def build_tree(x_vals, y_vals):
    print("Building node..")
    base_node = Node()
    min_attr = -1
    min_entr = -1
    split_val = -1
    print(x_vals[0][0])
    print(len(x_vals[0]))
    print(str(range(len(x_vals[0]))))
    for i in range(len(x_vals[0])):
        print(str(i) + ", ")
        possible_attrs = set(get_col(x_vals, i))
        for possibility in possible_attrs:
            curr_entr = entropy(x_vals, y_vals, i, possibility)
            if min_attr == -1 or min_entr == -1 or split_val == -1:
                min_attr = i
                min_entr = curr_entr
                split_val = possibility
            elif curr_entr < min_entr:
                min_attr = i
                min_entr = curr_entr
                split_val = possibility
    print("Results: min_attr=" + str(min_attr) + ", min_entr=" + str(min_entr) + ", split_val=" + str(split_val))
    if min_entr < 0.01 or len(y_vals) < 2:
        return get_dominant_val(y_vals)
    else:
        base_node.attribute = min_attr
        base_node.split_value = split_val
        right_x, left_x, right_y, left_y = split_attr(x_vals, y_vals, min_attr, split_val)
        base_node.left_branch = build_tree(left_x, left_y)
        base_node.right_branch = build_tree(right_x, right_y)
        return base_node

In [46]:
base_node = build_tree(votes_train_x, votes_train_y)

Building node..
n
16
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
0, 
Ratio: 0.184210526316
Ratio: 0.822580645161
Ratio: 0.0161290322581
Ratio: 0.815789473684
Ratio: 0.161290322581
1, 
Ratio: 0.684210526316
Ratio: 0.629032258065
Ratio: 0.0789473684211
Ratio: 0.0483870967742
Ratio: 0.236842105263
Ratio: 0.322580645161
2, 
Ratio: 0.0526315789474
Ratio: 0.951612903226
Ratio: 0.0161290322581
Ratio: 0.947368421053
Ratio: 0.0322580645161
3, 
Ratio: 1.0
Ratio: 0.0483870967742
Ratio: 0.0161290322581
Ratio: 0.935483870968
4, 
Ratio: 0.973684210526
Ratio: 0.225806451613
Ratio: 0.0161290322581
Ratio: 0.0263157894737
Ratio: 0.758064516129
5, 
Ratio: 0.868421052632
Ratio: 0.338709677419
Ratio: 0.0322580645161
Ratio: 0.131578947368
Ratio: 0.629032258065
6, 
Ratio: 0.131578947368
Ratio: 0.758064516129
Ratio: 0.0161290322581
Ratio: 0.868421052632
Ratio: 0.225806451613
7, 
Ratio: 0.0526315789474
Ratio: 0.758064516129
Ratio: 0.0263157894737
Ratio: 0.0161290322581
Ratio: 0.921052631579
Ratio: 0

In [47]:
def printtree(tree,indent='\t'):
   # Is this a leaf node?
   if not isinstance(tree, Node):
       print str(tree).upper()
   else:      
       # Print the criteria
       print 'Column : ' + str(tree.attribute) + " Value: " + str(tree.split_value)
       # Print the branches
       print indent+'True->',
       printtree(tree.right_branch,indent+'\t')
       print indent+'False->',
       printtree(tree.left_branch,indent+'  ')

printtree(base_node)

Column : 3 Value: n
	True-> DEMOCRAT
	False-> Column : 4 Value: y
	  True-> REPUBLICAN
	  False-> REPUBLICAN


In [48]:
print("Prediction: " + base_node.predict(votes_test_x[3]))
print("Actual: " + votes_test_y[3][0])

Prediction: democrat
Actual: democrat


In [49]:
def get_accuracy(tree, x_test, y_test):
    total = len(y_test)
    correct = 0.0
    for i in range(total):
        if tree.predict(x_test[i]) == y_test[i][0]:
            correct += 1.0
    return correct / total

In [50]:
print("Accuracy: " + str(get_accuracy(base_node, votes_test_x, votes_test_y)))

Accuracy: 0.940298507463
